### 5.BERT Fine-tuning for Question and Answering

In [3]:
# 라이브러리
import torch
from transformers import BertForQuestionAnswering, BertTokenizer

- 여기서는 스탠포드 질문-응답 데이터셋(SQUAD)을 기반으로 파인튜닝된 bert-large-uncased-whole-word-masking-finetuned-squad를 사용한다.

In [2]:
# 모델 불러오기
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

- 모델과 토크나이저를 불러왔다.
- 이제 입력을 전처리하자.

In [15]:
# 질문 및 단락
question = "What is the immune system?"
paragraph = "The immune system is a system of many biological structures and processes within an organism that protects against disease. To function properly, an immune system must detect a wide variety of agents, known as pathogens, from viruses to parasitic worms, and distinguish them from the organism's own healthy tissue."

In [16]:
# 특수토큰 추가
question = '[CLS]' + question + '[SEP]'
paragraph = paragraph + '[SEP]'

In [17]:
# 질문 및 단락 토큰화
question_tokens = tokenizer.tokenize(question)
paragraph_tokens = tokenizer.tokenize(paragraph)

In [18]:
# ID로 변환
tokens = question_tokens + paragraph_tokens
input_ids = tokenizer.convert_tokens_to_ids(tokens)

In [19]:
# 세그먼트 ID 정의
segment_ids = [0] * len(question_tokens)
segment_ids += [1] * len(paragraph_tokens)

- 질문과 단락을 구분하기 위해 질문은 0, 단락은 1로 분리한다.

In [20]:
# 텐서로 변환
input_ids = torch.tensor([input_ids])
segment_ids = torch.tensor([segment_ids])

입력에 대한 모든 처리를 완료했다. 이제 시작 점수와 끝 점수를 반환하는 모델에 텐서로 변환한 데이터를 입력하자.

In [23]:
scores = model(input_ids, token_type_ids = segment_ids)

- 이제 각각 확률값이 가장 큰 시작/끝 벡터를 추출한다.

In [24]:
# 가장 큰 확률 인덱스 추출
start_index = torch.argmax(scores.start_logits)
end_index = torch.argmax(scores.end_logits)

In [25]:
# 답변 확인
print(' '.join(tokens[start_index:end_index+1]))

a system of many biological structures and processes within an organism that protects against disease
